In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Clustering

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn for clustering and preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# For visualization
%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
# Define the dataset path
dataset_filepath = '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/all_books_all_topics.csv'

In [ ]:
# Load the dataset
df = pd.read_csv(dataset_filepath)

In [ ]:
# Display the first few rows
print("First 5 rows of the dataset:")
display(df.head())

First 5 rows of the dataset:


,Book Title,Topic,Topic_Label,Probability
0,A Long Time Coming,Topic_0,Expressions of Joy and Amusement,79.474938
1,A Not So Meet Cute,Topic_0,Expressions of Joy and Amusement,71.655854
2,Ache for You,Topic_0,Expressions of Joy and Amusement,56.319151
3,An Accidental Date with a Billionaire,Topic_0,Expressions of Joy and Amusement,26.288002
4,Bad Boss,Topic_0,Expressions of Joy and Amusement,43.874309


In [ ]:
# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())


Missing values in each column:
Book Title     0
Topic          0
Topic_Label    0
Probability    0
dtype: int64


In [ ]:
# Summary statistics
print("\nSummary statistics:")
display(df.describe())

# Verify the number of unique topics
unique_topics = df['Topic'].nunique()
print(f"Number of unique topics in the dataset: {unique_topics}")


Summary statistics:


,Probability
count,22575.000000
mean,29.854139
std,16.136130
min,0.700859
25%,18.110644
50%,27.829047
75%,39.048450
max,114.721734


Number of unique topics in the dataset: 215


In [ ]:
# Create a mapping from 'Topic' string to 'Topic_Label'
topic_labels_df = df[['Topic', 'Topic_Label']].drop_duplicates().sort_values('Topic')

# Create the mapping dictionary with 'Topic' strings as keys
topic_labels = topic_labels_df.set_index('Topic')['Topic_Label'].to_dict()

# Print the first 10 topic mappings to verify
print("Topic number to Topic_Label mapping (first 10 topics):")
for topic, label in list(topic_labels.items())[:10]:
    print(f"{topic}: {label}")

Topic number to Topic_Label mapping (first 10 topics):
Topic_0: Expressions of Joy and Amusement
Topic_1: Dining and Meals
Topic_10: Gazes and Eye Contact
Topic_100: Clocks and Timekeeping
Topic_101: Jealousy and Envy
Topic_102: Pleasure and Seduction
Topic_103: Education and School
Topic_104: Ties and Formal Wear
Topic_105: Kneeling and Lower Body Actions
Topic_106: Tone and Communication


In [ ]:
# Verify that all topics in df_pivot are present in topic_labels
# (We'll define df_pivot in the next cell, but to prepare, extract all unique topics)
unique_topics_in_df = df['Topic'].unique()

# Identify any topics not present in topic_labels
missing_in_mapping = set(unique_topics_in_df) - set(topic_labels.keys())

if missing_in_mapping:
    print(f"Missing topics in mapping: {missing_in_mapping}")
    # Optionally, add placeholder labels for missing topics
    for missing_topic in missing_in_mapping:
        topic_labels[missing_topic] = "Unknown Topic"
    print(f"Added placeholder labels for missing topics: {missing_in_mapping}")
else:
    print("All topics are present in the mapping.")

All topics are present in the mapping.


In [ ]:
# Pivot the DataFrame to have books as rows and topics as columns
df_pivot = df.pivot_table(
    index='Book Title',          # Corrected column name
    columns='Topic',
    values='Probability',
    fill_value=0
)

# Display the pivoted DataFrame
print("Pivoted DataFrame (df_pivot):")
display(df_pivot.head())

# Display the shape of the pivoted DataFrame
print(f"{df_pivot.shape[0]} rows × {df_pivot.shape[1]} columns")

Pivoted DataFrame (df_pivot):


Topic,Topic_0,Topic_1,Topic_10,Topic_100,Topic_101,Topic_102,Topic_103,Topic_104,Topic_105,Topic_106,...,Topic_90,Topic_91,Topic_92,Topic_93,Topic_94,Topic_95,Topic_96,Topic_97,Topic_98,Topic_99
Book Title,,,,,,,,,,,,,,,,,,,,,
A Long Time Coming,79.474938,56.317327,69.781076,28.041094,62.643593,86.409227,41.159529,44.710135,44.032778,65.074435,...,37.320165,28.184105,19.073854,50.570807,49.238223,45.548591,40.048028,50.409369,68.443000,75.107108
A Not So Meet Cute,71.655854,51.052766,63.149428,22.492508,56.784662,77.896244,37.809100,40.955683,39.315136,58.919767,...,30.414757,27.199027,19.443791,44.797097,44.021191,40.037086,34.680983,45.963807,58.682914,62.849054
Ache for You,56.319151,38.191399,51.023719,19.264886,45.389982,58.180555,25.342493,35.025604,32.546548,47.054189,...,26.805348,24.507383,18.228584,34.295827,36.229980,34.181295,27.437485,35.395689,43.722991,48.661635
An Accidental Date with a Billionaire,26.288002,18.064906,23.180115,8.502990,21.700494,26.564325,12.985737,15.524906,14.501216,20.750922,...,9.669584,9.670462,6.813146,15.031829,16.342032,14.682329,11.619466,14.573176,21.545200,21.799730
Bad Boss,43.874309,29.619211,40.246871,17.512199,33.221254,45.192322,24.778549,26.592173,24.266978,34.811752,...,18.653640,18.565761,13.603523,28.130165,28.270488,24.141426,21.893897,25.668722,38.207716,37.302531


105 rows × 215 columns


In [ ]:
# Function to get top N topics per cluster
def get_top_topics(cluster_data, topic_labels, top_n=30):
    """
    Returns a DataFrame of top N topics based on mean probability within the cluster.
    """
    # Calculate mean probability per topic within the cluster
    topic_means = cluster_data.mean().sort_values(ascending=False).head(top_n)
    topic_std = cluster_data.std().loc[topic_means.index]
    top_topics_df = pd.DataFrame({
        'Topic': topic_means.index,
        'Topic_Label': topic_means.index.map(topic_labels),
        'Mean_Probability': topic_means.values,
        'Std_Deviation': topic_std.values
    })
    return top_topics_df

# Function to get all topics per cluster
def get_all_topics(cluster_data, topic_labels):
    """
    Returns a DataFrame of all topics with their mean probabilities and standard deviations within the cluster.
    """
    topic_means = cluster_data.mean().sort_values(ascending=False)
    topic_std = cluster_data.std().loc[topic_means.index]
    all_topics_df = pd.DataFrame({
        'Topic': topic_means.index,
        'Topic_Label': topic_means.index.map(topic_labels),
        'Mean_Probability': topic_means.values,
        'Std_Deviation': topic_std.values
    })
    return all_topics_df

# Function to get top N books per cluster
def get_top_books(cluster_data, top_n=5):
    """
    Returns a list of top N book titles based on the sum of their topic probabilities within the cluster.
    """
    # Sum probabilities across all topics for each book to determine top books
    top_books = cluster_data.sum(axis=1).sort_values(ascending=False).head(top_n).index.tolist()
    return top_books

# Function to get all books sorted by sum of topic probabilities
def get_all_books(cluster_data):
    """
    Returns a DataFrame of all books sorted by the sum of their topic probabilities within the cluster.
    """
    book_sums = cluster_data.sum(axis=1).sort_values(ascending=False)
    all_books_df = pd.DataFrame({
        'Book Title': book_sums.index,
        'Total_Probability': book_sums.values
    })
    return all_books_df

In [ ]:
# Verification of helper functions with sample data
print("\nVerifying helper functions with sample data...")

# Create a sample cluster_data by selecting a subset of df_pivot
sample_cluster_data = df_pivot.iloc[:10]  # taking first 10 books as sample

# Get top 30 topics for sample data
sample_top_topics = get_top_topics(sample_cluster_data, topic_labels, top_n=30)
print("\nSample Top 30 Topics:")
display(sample_top_topics)

# Get all topics for sample data
sample_all_topics = get_all_topics(sample_cluster_data, topic_labels)
print("\nSample All Topics:")
display(sample_all_topics)

# Get top 5 books for sample data
sample_top_books = get_top_books(sample_cluster_data, top_n=5)
print("\nSample Top 5 Books:")
for i, book in enumerate(sample_top_books, 1):
    print(f"{i}. {book}")

# Get all books sorted by total probability
sample_all_books = get_all_books(sample_cluster_data)
print("\nSample All Books Sorted by Total Probability:")
display(sample_all_books.head())


Verifying helper functions with sample data...

Sample Top 30 Topics:


,Topic,Topic_Label,Mean_Probability,Std_Deviation
0,Topic_72,Love and Affection,59.046471,16.735448
1,Topic_70,Leaving and Letting Go,57.380120,17.692158
2,Topic_201,Happiness and Gratitude,56.663655,17.409058
3,Topic_102,Pleasure and Seduction,56.006795,17.780864
4,Topic_175,Moments and Realizations,54.540264,16.647522
5,Topic_129,Breaking and Letting Go,53.401722,16.719943
6,Topic_0,Expressions of Joy and Amusement,53.294888,15.682545
7,Topic_117,Possession and Desire,53.048371,19.880187
8,Topic_36,Nods and Agreement,52.742087,15.089946
9,Topic_12,Nightlife and Events,51.614334,15.887609



Sample All Topics:


,Topic,Topic_Label,Mean_Probability,Std_Deviation
0,Topic_72,Love and Affection,59.046471,16.735448
1,Topic_70,Leaving and Letting Go,57.380120,17.692158
2,Topic_201,Happiness and Gratitude,56.663655,17.409058
3,Topic_102,Pleasure and Seduction,56.006795,17.780864
4,Topic_175,Moments and Realizations,54.540264,16.647522
...,...,...,...,...
210,Topic_92,Mirrors and Reflections,15.044534,4.108528
211,Topic_114,Screens and Viewing,13.472833,4.658705
212,Topic_151,Buttons and Devices,12.359829,4.201884
213,Topic_186,Laptops and Technology,8.218655,2.578650



Sample Top 5 Books:
1. A Long Time Coming
2. A Not So Meet Cute
3. Banking the Billionaire
4. Ache for You
5. Between Commitment and Betrayal

Sample All Books Sorted by Total Probability:


,Book Title,Total_Probability
0,A Long Time Coming,10636.420790
1,A Not So Meet Cute,9423.447186
2,Banking the Billionaire,8288.035902
3,Ache for You,7527.456728
4,Between Commitment and Betrayal,7343.206730


In [ ]:
# Ensure that there are no NaN values in 'Topic_Label' columns
# Select all unique topics from df_pivot
pivot_topics = df_pivot.columns.tolist()

# Check for any topics without a corresponding label
topics_without_labels = [topic for topic in pivot_topics if topic not in topic_labels]

if topics_without_labels:
    print(f"Topics without labels: {topics_without_labels}")
    # Optionally, add placeholder labels
    for topic in topics_without_labels:
        topic_labels[topic] = "Unknown Topic"
    print(f"Added placeholder labels for missing topics: {topics_without_labels}")
else:
    print("All topics have corresponding labels.")

All topics have corresponding labels.


In [ ]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Dropped {duplicates} duplicate rows.")
else:
    print("No duplicate rows found.")

Number of duplicate rows: 0
No duplicate rows found.


In [ ]:
# Downcast numerical columns to save memory
df['Probability'] = pd.to_numeric(df['Probability'], downcast='float')

# Convert 'Book Title' and 'Topic' columns to categorical types if there are many repeated values
df['Book Title'] = df['Book Title'].astype('category')
df['Topic'] = df['Topic'].astype('category')

print("Data types after optimization:")
print(df.dtypes)

Data types after optimization:
Book Title     category
Topic          category
Topic_Label      object
Probability     float32
dtype: object


In [ ]:
# Inspect the shape of df_pivot
print(f"Shape of df_pivot: {df_pivot.shape}")

# Inspect column names
print("\nColumns in df_pivot:")
print(df_pivot.columns.tolist())

# Inspect a sample of df_pivot
print("\nSample data from df_pivot:")
display(df_pivot.head())

Shape of df_pivot: (105, 215)

Columns in df_pivot:
['Topic_0', 'Topic_1', 'Topic_10', 'Topic_100', 'Topic_101', 'Topic_102', 'Topic_103', 'Topic_104', 'Topic_105', 'Topic_106', 'Topic_107', 'Topic_108', 'Topic_109', 'Topic_11', 'Topic_110', 'Topic_111', 'Topic_112', 'Topic_113', 'Topic_114', 'Topic_115', 'Topic_116', 'Topic_117', 'Topic_118', 'Topic_119', 'Topic_12', 'Topic_120', 'Topic_121', 'Topic_122', 'Topic_123', 'Topic_124', 'Topic_125', 'Topic_126', 'Topic_127', 'Topic_128', 'Topic_129', 'Topic_13', 'Topic_130', 'Topic_131', 'Topic_132', 'Topic_133', 'Topic_134', 'Topic_135', 'Topic_136', 'Topic_137', 'Topic_138', 'Topic_139', 'Topic_14', 'Topic_140', 'Topic_141', 'Topic_142', 'Topic_143', 'Topic_144', 'Topic_145', 'Topic_146', 'Topic_147', 'Topic_148', 'Topic_149', 'Topic_15', 'Topic_150', 'Topic_151', 'Topic_152', 'Topic_153', 'Topic_154', 'Topic_155', 'Topic_156', 'Topic_157', 'Topic_158', 'Topic_159', 'Topic_16', 'Topic_160', 'Topic_161', 'Topic_162', 'Topic_163', 'Topic_16

Topic,Topic_0,Topic_1,Topic_10,Topic_100,Topic_101,Topic_102,Topic_103,Topic_104,Topic_105,Topic_106,...,Topic_90,Topic_91,Topic_92,Topic_93,Topic_94,Topic_95,Topic_96,Topic_97,Topic_98,Topic_99
Book Title,,,,,,,,,,,,,,,,,,,,,
A Long Time Coming,79.474938,56.317327,69.781076,28.041094,62.643593,86.409227,41.159529,44.710135,44.032778,65.074435,...,37.320165,28.184105,19.073854,50.570807,49.238223,45.548591,40.048028,50.409369,68.443000,75.107108
A Not So Meet Cute,71.655854,51.052766,63.149428,22.492508,56.784662,77.896244,37.809100,40.955683,39.315136,58.919767,...,30.414757,27.199027,19.443791,44.797097,44.021191,40.037086,34.680983,45.963807,58.682914,62.849054
Ache for You,56.319151,38.191399,51.023719,19.264886,45.389982,58.180555,25.342493,35.025604,32.546548,47.054189,...,26.805348,24.507383,18.228584,34.295827,36.229980,34.181295,27.437485,35.395689,43.722991,48.661635
An Accidental Date with a Billionaire,26.288002,18.064906,23.180115,8.502990,21.700494,26.564325,12.985737,15.524906,14.501216,20.750922,...,9.669584,9.670462,6.813146,15.031829,16.342032,14.682329,11.619466,14.573176,21.545200,21.799730
Bad Boss,43.874309,29.619211,40.246871,17.512199,33.221254,45.192322,24.778549,26.592173,24.266978,34.811752,...,18.653640,18.565761,13.603523,28.130165,28.270488,24.141426,21.893897,25.668722,38.207716,37.302531


In [ ]:
# Convert df_pivot to a sparse DataFrame if it contains many zeros
df_pivot_sparse = df_pivot.astype(pd.SparseDtype("float", 0))

# Display information about the sparse DataFrame
print("Sparse DataFrame Info:")
df_pivot_sparse.info()

Sparse DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, A Long Time Coming to Unmasking the Billionaire
Columns: 215 entries, Topic_0 to Topic_99
dtypes: Sparse[float64, 0](215)
memory usage: 265.4+ KB


In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Select topic columns (all columns in df_pivot)
topic_columns = df_pivot.columns.tolist()

# Fit and transform the data
df_scaled = scaler.fit_transform(df_pivot)

# Convert the scaled data back to a DataFrame
df_scaled = pd.DataFrame(df_scaled, index=df_pivot.index, columns=topic_columns)

print("Data scaling completed. Here's a sample of the scaled data:")
display(df_scaled.head())

Data scaling completed. Here's a sample of the scaled data:


,Topic_0,Topic_1,Topic_10,Topic_100,Topic_101,Topic_102,Topic_103,Topic_104,Topic_105,Topic_106,...,Topic_90,Topic_91,Topic_92,Topic_93,Topic_94,Topic_95,Topic_96,Topic_97,Topic_98,Topic_99
Book Title,,,,,,,,,,,,,,,,,,,,,
A Long Time Coming,1.515081,1.642712,1.454704,1.516337,1.492844,1.665690,1.557965,1.324266,1.424637,1.523449,...,1.631003,1.030558,0.801507,1.634371,1.345860,1.408287,1.576822,1.661021,1.679045,1.820568
A Not So Meet Cute,1.136348,1.268476,1.090310,0.759248,1.130412,1.267855,1.233424,1.016844,1.031194,1.168156,...,0.940151,0.911657,0.861984,1.178393,0.956806,0.967622,1.058646,1.316378,1.099651,1.149389
Ache for You,0.393485,0.354212,0.424030,0.318848,0.425540,0.346485,0.025838,0.531278,0.466706,0.483187,...,0.579048,0.586771,0.663324,0.349056,0.375786,0.499430,0.359302,0.497081,0.211576,0.372571
An Accidental Date with a Billionaire,-1.061133,-1.076500,-1.105912,-1.149582,-1.039885,-1.131030,-1.171107,-1.065478,-1.038242,-1.035232,...,-1.135300,-1.204073,-1.202854,-1.172317,-1.107333,-1.059584,-1.167893,-1.117191,-1.104979,-1.098227
Bad Boss,-0.209305,-0.255150,-0.168133,0.079699,-0.327214,-0.260491,-0.028789,-0.159268,-0.223795,-0.223537,...,-0.236490,-0.130394,-0.092774,-0.137877,-0.217783,-0.303294,-0.175920,-0.257005,-0.115831,-0.249386


In [ ]:
# Define the output directory
output_dir = '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print(f"Output directory set to: {output_dir}")

Output directory set to: /content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs


In [ ]:
# Define the number of clusters to analyze
cluster_range = [3, 4, 5]

for k in cluster_range:
    print(f"\nPerforming K-Means clustering with k={k}...")

    # Initialize and fit KMeans
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(df_scaled)

    # Assign cluster labels to df_pivot
    df_pivot[f'Cluster_{k}'] = cluster_labels

    # Create a mapping from Book Title to Cluster
    cluster_map = df_pivot[f'Cluster_{k}'].to_dict()

    # Assign clusters to the original df based on Book Title
    df[f'Cluster_{k}'] = df['Book Title'].map(cluster_map)

    # Verify the mapping by displaying a sample
    print(f"Assigned clusters to the original DataFrame for k={k}.")
    print(f"Sample cluster assignments for k={k}:")
    display(df[['Book Title', f'Cluster_{k}']].drop_duplicates().head())

    # Prepare to store results
    top_topics_all_clusters = {}
    all_topics_all_clusters = {}
    top_books_all_clusters = {}
    all_books_all_clusters = {}
    cluster_profiles = {}
    thematic_spread = {}

    # Analyze each cluster
    for cluster in range(k):
        print(f"\nAnalyzing Cluster {cluster} for k={k}...")

        # Filter data for the current cluster from df_pivot
        cluster_books = df_pivot[df_pivot[f'Cluster_{k}'] == cluster]

        # Define topic columns as columns starting with 'Topic_'
        topic_columns_cluster = [col for col in cluster_books.columns if col.startswith('Topic_')]

        # Check if topic_columns_cluster are present
        if not topic_columns_cluster:
            print(f"Warning: No topic columns found for Cluster {cluster} in k={k}.")
            continue  # Skip to next cluster

        # Select only topic columns
        cluster_books_topic = cluster_books[topic_columns_cluster]

        # Get top 30 topics
        top_topics_df = get_top_topics(cluster_books_topic, topic_labels, top_n=30)
        top_topics_all_clusters[cluster] = top_topics_df

        # Get all topics
        all_topics_df = get_all_topics(cluster_books_topic, topic_labels)
        all_topics_all_clusters[cluster] = all_topics_df

        # Get top 5 books
        top_books = get_top_books(cluster_books_topic, top_n=5)
        top_books_all_clusters[cluster] = top_books

        # Get all books sorted by total probability
        all_books_df = get_all_books(cluster_books_topic)
        all_books_all_clusters[cluster] = all_books_df

        # Cluster Size and Percentage
        cluster_size = cluster_books.shape[0]
        cluster_percentage = (cluster_size / df_pivot.shape[0]) * 100

        # Cluster Profiles with Statistical Measures
        cluster_profiles[cluster] = {
            'Cluster_Size': cluster_size,
            'Cluster_Percentage': cluster_percentage,
            'Top_Topics': top_topics_df
        }

        # Thematic Spread
        # For dominant themes (top 30 topics), calculate min, max, median probabilities
        dominant_topics = top_topics_df['Topic'].tolist()
        # Ensure that dominant_topics exist in cluster_books_topic
        existing_dominant_topics = [topic for topic in dominant_topics if topic in cluster_books_topic.columns]
        if not existing_dominant_topics:
            print(f"Warning: No dominant topics found for Cluster {cluster} in k={k}.")
            continue
        dominant_data = cluster_books_topic[existing_dominant_topics]

        # Check if dominant_data is empty
        if dominant_data.empty:
            print(f"Warning: dominant_data is empty for Cluster {cluster} in k={k}.")
            continue

        # Debugging: Print head of dominant_data
        print(f"Head of dominant_data for Cluster {cluster} in k={k}:")
        display(dominant_data.head())

        # Calculate Probability Range and Std Deviation using separate aggregations
        prob_min = dominant_data.min().to_dict()
        prob_max = dominant_data.max().to_dict()
        prob_median = dominant_data.median().to_dict()

        prob_range = {
            'min': prob_min,
            'max': prob_max,
            'median': prob_median
        }

        std_deviation = dominant_data.std().to_dict()

        # Debugging: Print probability_range structure
        print(f"Probability_Range for Cluster {cluster} in k={k}:")
        print(prob_range)

        thematic_spread[cluster] = {
            'Probability_Range': prob_range,
            'Std_Deviation_per_Topic': std_deviation
        }

    # Save Top 30 Topics and All Topics per Cluster to CSV
    for cluster in range(k):
        if cluster not in top_topics_all_clusters or cluster not in all_topics_all_clusters:
            print(f"Skipping saving topics for Cluster {cluster} in k={k} due to missing data.")
            continue

        # Top 30 Topics
        top_topics_df = top_topics_all_clusters[cluster]
        top_topics_output = os.path.join(output_dir, f'top_30_topics_k{k}_cluster{cluster}.csv')
        top_topics_df.to_csv(top_topics_output, index=False)
        print(f"Saved top 30 topics for Cluster {cluster} to '{top_topics_output}'.")

        # All Topics
        all_topics_df = all_topics_all_clusters[cluster]
        all_topics_output = os.path.join(output_dir, f'all_topics_k{k}_cluster{cluster}.csv')
        all_topics_df.to_csv(all_topics_output, index=False)
        print(f"Saved all topics for Cluster {cluster} to '{all_topics_output}'.")

    # Save Top 5 Books and All Books per Cluster to CSV
    for cluster in range(k):
        if cluster not in top_books_all_clusters or cluster not in all_books_all_clusters:
            print(f"Skipping saving books for Cluster {cluster} in k={k} due to missing data.")
            continue

        # Top 5 Books
        top_books = top_books_all_clusters[cluster]
        top_books_df = pd.DataFrame({'Book Title': top_books})
        top_books_output = os.path.join(output_dir, f'top_5_books_k{k}_cluster{cluster}.csv')
        top_books_df.to_csv(top_books_output, index=False)
        print(f"Saved top 5 books for Cluster {cluster} to '{top_books_output}'.")

        # All Books
        all_books_df = all_books_all_clusters[cluster]
        all_books_output = os.path.join(output_dir, f'all_books_k{k}_cluster{cluster}.csv')
        all_books_df.to_csv(all_books_output, index=False)
        print(f"Saved all books for Cluster {cluster} to '{all_books_output}'.")

    # Save Cluster Profiles with Statistical Measures
    profiles_data = []
    for cluster, profile in cluster_profiles.items():
        size = profile['Cluster_Size']
        percentage = profile['Cluster_Percentage']
        for _, row in profile['Top_Topics'].iterrows():
            profiles_data.append({
                'Cluster': cluster,
                'Topic': row['Topic'],
                'Topic_Label': row['Topic_Label'],
                'Mean_Probability': row['Mean_Probability'],
                'Std_Deviation': row['Std_Deviation'],
                'Cluster_Size': size,
                'Cluster_Percentage': percentage
            })
    profiles_df = pd.DataFrame(profiles_data)
    profiles_output = os.path.join(output_dir, f'cluster_profiles_k{k}.csv')
    profiles_df.to_csv(profiles_output, index=False)
    print(f"Saved cluster profiles for k={k} to '{profiles_output}'.")

    # Save Thematic Spread
    thematic_data = []
    for cluster, spread in thematic_spread.items():
        prob_range = spread.get('Probability_Range', {})
        std_dev = spread.get('Std_Deviation_per_Topic', {})

        # Ensure 'min', 'max', 'median' keys exist
        for key in ['min', 'max', 'median']:
            if key not in prob_range:
                prob_range[key] = {}
                print(f"Warning: '{key}' key missing in Probability_Range for Cluster {cluster}. Assigning NaN.")

        for topic, std in std_dev.items():
            min_prob = prob_range['min'].get(topic, float('nan'))
            max_prob = prob_range['max'].get(topic, float('nan'))
            median_prob = prob_range['median'].get(topic, float('nan'))

            thematic_data.append({
                'Cluster': cluster,
                'Topic': topic,
                'Min_Probability': min_prob,
                'Max_Probability': max_prob,
                'Median_Probability': median_prob,
                'Std_Deviation': std
            })

    thematic_df = pd.DataFrame(thematic_data)
    thematic_output = os.path.join(output_dir, f'thematic_spread_k{k}.csv')
    thematic_df.to_csv(thematic_output, index=False)
    print(f"Saved thematic spread for k={k} to '{thematic_output}'.")

    # Save Cluster Assignments
    cluster_assignments_output = os.path.join(output_dir, f'cluster_assignments_k{k}.csv')
    cluster_assignments = df_pivot[[f'Cluster_{k}']].rename(columns={f'Cluster_{k}': 'Cluster'})
    cluster_assignments.to_csv(cluster_assignments_output)
    print(f"Saved cluster assignments for k={k} to '{cluster_assignments_output}'.")

    print(f"Clustering with k={k} completed and results saved.\n")


Performing K-Means clustering with k=3...
Assigned clusters to the original DataFrame for k=3.
Sample cluster assignments for k=3:


,Book Title,Cluster_3
0,A Long Time Coming,1
1,A Not So Meet Cute,1
2,Ache for You,0
3,An Accidental Date with a Billionaire,2
4,Bad Boss,0



Analyzing Cluster 0 for k=3...
Head of dominant_data for Cluster 0 in k=3:


Topic,Topic_72,Topic_70,Topic_201,Topic_102,Topic_175,Topic_129,Topic_0,Topic_117,Topic_36,Topic_12,...,Topic_60,Topic_99,Topic_31,Topic_50,Topic_11,Topic_64,Topic_179,Topic_49,Topic_98,Topic_115
Book Title,,,,,,,,,,,,,,,,,,,,,
Ache for You,57.628415,56.880138,55.068065,58.180555,54.437852,55.985111,56.319151,56.039241,51.501959,51.596241,...,47.917208,48.661635,49.415486,46.686634,49.823008,49.887579,46.727125,45.559387,43.722991,45.005019
Bad Boss,48.937849,47.072433,48.225659,45.192322,47.101467,43.017024,43.874309,43.134256,44.232852,42.895156,...,36.850456,37.302531,37.725331,37.322676,36.370452,37.592128,37.788813,37.366143,38.207716,35.421537
Banking the Billionaire,67.338625,65.578731,66.372643,65.588555,63.678955,60.742803,63.634949,64.081367,61.743543,61.227850,...,53.324409,55.598779,51.983287,54.627552,54.169766,53.932228,52.671945,53.124814,52.618480,49.379849
Beauty and the Billionaire,58.845881,52.948413,53.467553,53.812438,51.278201,49.456941,52.195326,45.897764,50.635772,49.541668,...,45.127404,42.315750,44.239503,43.384968,42.107857,42.761932,42.097425,45.074004,40.805066,38.788087
Bedding the Billionaire,53.682072,50.571483,50.854971,49.483791,47.392565,46.359166,48.162375,41.062011,49.871963,46.706812,...,42.762807,39.776318,40.737828,41.861483,38.493562,40.962020,39.469905,42.749482,38.119258,37.274192


Probability_Range for Cluster 0 in k=3:
{'min': {'Topic_72': 42.24676432563, 'Topic_70': 42.07928208484, 'Topic_201': 41.23106852018, 'Topic_102': 41.48197991863, 'Topic_175': 39.7483221089, 'Topic_129': 39.0248389989, 'Topic_0': 39.61140213772, 'Topic_117': 37.59152399045, 'Topic_36': 38.2636188118, 'Topic_12': 38.53207717847, 'Topic_14': 37.7853262093, 'Topic_167': 34.32953572492, 'Topic_178': 35.66898896265, 'Topic_51': 33.93020488013, 'Topic_157': 34.71102750189, 'Topic_83': 34.84466220858, 'Topic_10': 35.444119069, 'Topic_84': 33.79986441381, 'Topic_187': 33.6999010024, 'Topic_119': 33.65291764828, 'Topic_60': 31.8313308354, 'Topic_99': 34.1156842785, 'Topic_31': 33.8241092529, 'Topic_50': 32.95822593158, 'Topic_11': 33.76902511737, 'Topic_64': 33.6123365737, 'Topic_179': 32.91236002747, 'Topic_49': 31.54990131354, 'Topic_98': 32.4656066582, 'Topic_115': 31.04694880438}, 'max': {'Topic_72': 75.1360006544, 'Topic_70': 70.92149856687, 'Topic_201': 68.24951752723, 'Topic_102': 70.952

...

Probability_Range for Cluster 2 in k=5:
{'min': {'Topic_72': 6.6902022193, 'Topic_70': 5.96029475794, 'Topic_201': 6.0391171624, 'Topic_102': 7.2027280112, 'Topic_175': 5.8973335354, 'Topic_129': 5.875989118, 'Topic_36': 5.6227633015, 'Topic_0': 6.5209316176, 'Topic_167': 6.5901355198, 'Topic_12': 5.9276324582, 'Topic_117': 6.3728193862, 'Topic_14': 5.7331448988, 'Topic_51': 5.4680915485, 'Topic_157': 5.64153411886, 'Topic_83': 5.32849736646, 'Topic_10': 5.9203868564, 'Topic_60': 5.47602294465, 'Topic_178': 5.18784214015, 'Topic_187': 5.5543064092, 'Topic_84': 5.02738840396, 'Topic_31': 5.903578033, 'Topic_119': 5.5595529533, 'Topic_49': 5.2470400084, 'Topic_99': 5.20229171795, 'Topic_50': 5.12978856455, 'Topic_64': 5.65503210336, 'Topic_195': 4.3520545152, 'Topic_98': 4.77266021001, 'Topic_179': 5.2392391498, 'Topic_24': 5.25364928371}, 'max': {'Topic_72': 36.7621211663, 'Topic_70': 36.6796035038, 'Topic_201': 34.92051411986, 'Topic_102': 33.34814110219, 'Topic_175': 32.6473061453, 'T

Topic,Topic_117,Topic_102,Topic_0,Topic_70,Topic_201,Topic_72,Topic_129,Topic_175,Topic_14,Topic_36,...,Topic_99,Topic_68,Topic_51,Topic_179,Topic_157,Topic_73,Topic_31,Topic_106,Topic_52,Topic_133
Book Title,,,,,,,,,,,,,,,,,,,,,
The Billionaire's Christmas Bride,108.817997,107.057802,105.841517,102.955869,98.554079,98.020964,100.409149,98.985492,99.123465,97.316764,...,92.089320,92.596583,89.108042,89.264143,88.065528,91.285401,88.931692,90.284275,93.335450,88.106608
The Billionaire's Fake Wife,89.223509,86.520488,83.935525,84.386533,80.472241,79.632029,81.365516,82.271574,79.384505,79.076076,...,75.282352,75.638829,73.293841,72.559487,71.985576,72.124742,72.751718,72.762141,73.097412,71.741811
The Billionaire's Secret,114.721734,114.112702,110.743120,107.861760,103.110809,103.904531,106.219888,104.546611,102.716733,102.069216,...,94.768460,98.312488,94.768671,92.256756,93.379117,95.704003,95.701019,93.956020,97.612517,93.961619
The Stopover,98.855335,97.211921,98.803577,100.761172,98.892348,97.700626,94.681787,93.927488,96.476589,91.332969,...,80.962937,78.209747,80.664001,81.665243,81.124127,77.461875,77.865082,79.358589,74.005676,78.183528
The Takeover,100.133924,98.755546,100.231517,101.654020,101.054947,102.013082,97.187188,94.095525,95.666194,91.223609,...,80.699036,78.446376,80.938291,80.750235,81.414336,79.012580,80.054863,77.826849,74.444545,77.851128


Probability_Range for Cluster 3 in k=5:
{'min': {'Topic_117': 89.22350887143, 'Topic_102': 86.5204883794, 'Topic_0': 83.93552504704, 'Topic_70': 84.38653269954, 'Topic_201': 80.47224084414, 'Topic_72': 79.63202864223, 'Topic_129': 81.36551597921, 'Topic_175': 82.27157422965, 'Topic_14': 79.38450461529, 'Topic_36': 79.07607566407, 'Topic_12': 75.32274129505, 'Topic_178': 77.55937848559999, 'Topic_64': 79.27135050887, 'Topic_11': 74.7457025241, 'Topic_119': 76.12279174031, 'Topic_10': 76.11966118938, 'Topic_60': 75.97108893923, 'Topic_167': 71.47765543042, 'Topic_84': 72.68204003732, 'Topic_83': 74.35756479252, 'Topic_99': 75.28235164419, 'Topic_68': 75.6388288141, 'Topic_51': 73.29384125456, 'Topic_179': 72.55948656087, 'Topic_157': 71.98557558946, 'Topic_73': 72.12474245203, 'Topic_31': 72.75171797122, 'Topic_106': 72.76214059846, 'Topic_52': 73.09741174633, 'Topic_133': 71.74181118747}, 'max': {'Topic_117': 114.72173433419, 'Topic_102': 114.11270234707, 'Topic_0': 110.74312029796, 'To

Topic,Topic_72,Topic_70,Topic_201,Topic_102,Topic_175,Topic_129,Topic_0,Topic_117,Topic_36,Topic_12,...,Topic_60,Topic_99,Topic_31,Topic_50,Topic_49,Topic_11,Topic_64,Topic_179,Topic_98,Topic_115
Book Title,,,,,,,,,,,,,,,,,,,,,
Ache for You,57.628415,56.880138,55.068065,58.180555,54.437852,55.985111,56.319151,56.039241,51.501959,51.596241,...,47.917208,48.661635,49.415486,46.686634,45.559387,49.823008,49.887579,46.727125,43.722991,45.005019
Banking the Billionaire,67.338625,65.578731,66.372643,65.588555,63.678955,60.742803,63.634949,64.081367,61.743543,61.227850,...,53.324409,55.598779,51.983287,54.627552,53.124814,54.169766,53.932228,52.671945,52.618480,49.379849
Beauty and the Billionaire,58.845881,52.948413,53.467553,53.812438,51.278201,49.456941,52.195326,45.897764,50.635772,49.541668,...,45.127404,42.315750,44.239503,43.384968,45.074004,42.107857,42.761932,42.097425,40.805066,38.788087
Between Commitment and Betrayal,64.770876,64.379214,59.978047,58.486863,59.239506,58.328255,53.188920,57.262236,55.193112,54.731581,...,49.072724,49.019952,47.582130,48.647688,48.297869,45.241027,46.280517,46.980806,47.883278,49.470807
Between Love and Loathing,65.652357,63.428170,61.956840,59.624659,58.216367,57.747023,54.926813,58.457895,56.894002,56.389985,...,47.548566,48.351280,48.217503,49.756389,48.565946,45.199277,45.947559,48.128840,47.002614,50.208413


Probability_Range for Cluster 4 in k=5:
{'min': {'Topic_72': 51.78232845106, 'Topic_70': 51.97685243855, 'Topic_201': 49.37901058366, 'Topic_102': 51.21786653832, 'Topic_175': 49.75909426403, 'Topic_129': 48.0332429687, 'Topic_0': 48.98195218858, 'Topic_117': 42.61638962508, 'Topic_36': 47.31505044821, 'Topic_12': 46.1489227469, 'Topic_14': 46.52930518972, 'Topic_167': 45.1353655735, 'Topic_178': 42.04481353904, 'Topic_51': 44.0876115043, 'Topic_83': 44.31722859836, 'Topic_157': 42.49347559358, 'Topic_84': 42.25994866496, 'Topic_10': 45.00354813667, 'Topic_187': 40.97237416936, 'Topic_119': 43.01622704679, 'Topic_60': 42.13119810984, 'Topic_99': 40.56145178004, 'Topic_31': 42.21945500717, 'Topic_50': 40.1105315662, 'Topic_49': 40.83120101438, 'Topic_11': 41.29993854204, 'Topic_64': 41.2614351248, 'Topic_179': 41.26676127121, 'Topic_98': 40.1209810964, 'Topic_115': 38.14369913799}, 'max': {'Topic_72': 75.1360006544, 'Topic_70': 70.92149856687, 'Topic_201': 68.24951752723, 'Topic_102': 7

In [ ]:
# List all files in the output directory to verify saved results
print(f"\nFiles saved in the output directory '{output_dir}':")
for filename in os.listdir(output_dir):
    print(filename)


Files saved in the output directory '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs':
top_30_topics_k3_cluster0.csv
all_topics_k3_cluster0.csv
top_30_topics_k3_cluster1.csv
all_topics_k3_cluster1.csv
top_30_topics_k3_cluster2.csv
all_topics_k3_cluster2.csv
top_5_books_k3_cluster0.csv
all_books_k3_cluster0.csv
top_5_books_k3_cluster1.csv
all_books_k3_cluster1.csv
top_5_books_k3_cluster2.csv
all_books_k3_cluster2.csv
cluster_profiles_k3.csv
thematic_spread_k3.csv
cluster_assignments_k3.csv
top_30_topics_k4_cluster0.csv
all_topics_k4_cluster0.csv
top_30_topics_k4_cluster1.csv
all_topics_k4_cluster1.csv
top_30_topics_k4_cluster2.csv
all_topics_k4_cluster2.csv
top_30_topics_k4_cluster3.csv
all_topics_k4_cluster3.csv
top_5_books_k4_cluster0.csv
all_books_k4_cluster0.csv
top_5_books_k4_cluster1.csv
all_books_k4_cluster1.csv
top_5_books_k4_cluster2.csv
all_books_k4_cluster2.csv
top_5_books_k4_cluster3.csv
all_books_k4_cluster3.csv
cluster_profiles_k

In [ ]:
# Corrected Verification of Cluster Assignments Using Merge

for k in cluster_range:
    print(f"\nVerifying cluster assignments for k={k}...")

    # Merge df and df_pivot on 'Book Title' to align cluster labels
    merged_df = df.merge(
        df_pivot[[f'Cluster_{k}']].reset_index(),  # Reset index to have 'Book Title' as a column
        on='Book Title',
        how='left',
        suffixes=('', '_pivot')
    )

    # Check for any missing cluster assignments after merge
    missing_assignments = merged_df[f'Cluster_{k}_pivot'].isnull().sum()
    if missing_assignments > 0:
        print(f"Warning: {missing_assignments} books in df do not have cluster assignments in df_pivot for k={k}.")

    # Compare the cluster labels
    consistency = (merged_df[f'Cluster_{k}'] == merged_df[f'Cluster_{k}_pivot']).all()

    if consistency:
        print(f"✔️ Cluster assignments for k={k} are consistent between df_pivot and original df.")
    else:
        # Identify discrepancies
        discrepancies = merged_df[merged_df[f'Cluster_{k}'] != merged_df[f'Cluster_{k}_pivot']]
        print(f"❌ Cluster assignments for k={k} are NOT consistent between df_pivot and original df.")
        print(f"Number of discrepancies: {discrepancies.shape[0]}")
        print("Sample discrepancies:")
        display(discrepancies[['Book Title', f'Cluster_{k}', f'Cluster_{k}_pivot']].head())


Verifying cluster assignments for k=3...
✔️ Cluster assignments for k=3 are consistent between df_pivot and original df.

Verifying cluster assignments for k=4...
✔️ Cluster assignments for k=4 are consistent between df_pivot and original df.

Verifying cluster assignments for k=5...
✔️ Cluster assignments for k=5 are consistent between df_pivot and original df.


In [ ]:
# Check for duplicate 'Book Title's in df
duplicate_books_df = df[df.duplicated(subset=['Book Title'], keep=False)]
if not duplicate_books_df.empty:
    print("Duplicate 'Book Title's found in df:")
    display(duplicate_books_df.head())
else:
    print("No duplicate 'Book Title's found in df.")

# Check for duplicate 'Book Title's in df_pivot
duplicate_books_pivot = df_pivot[df_pivot.index.duplicated(keep=False)]
if not duplicate_books_pivot.empty:
    print("Duplicate 'Book Title's found in df_pivot:")
    display(duplicate_books_pivot.head())
else:
    print("No duplicate 'Book Title's found in df_pivot.")

Duplicate 'Book Title's found in df:


,Book Title,Topic,Topic_Label,Probability,Cluster_3,Cluster_4,Cluster_5
0,A Long Time Coming,Topic_0,Expressions of Joy and Amusement,79.474937,1,1,1
1,A Not So Meet Cute,Topic_0,Expressions of Joy and Amusement,71.655853,1,1,1
2,Ache for You,Topic_0,Expressions of Joy and Amusement,56.319153,0,1,4
3,An Accidental Date with a Billionaire,Topic_0,Expressions of Joy and Amusement,26.288002,2,2,2
4,Bad Boss,Topic_0,Expressions of Joy and Amusement,43.874310,0,0,0


No duplicate 'Book Title's found in df_pivot.


In [ ]:
# Verify all expected output files are present
expected_files = []
for k in cluster_range:
    for cluster in range(k):
        expected_files.extend([
            f'top_30_topics_k{k}_cluster{cluster}.csv',
            f'all_topics_k{k}_cluster{cluster}.csv',
            f'top_5_books_k{k}_cluster{cluster}.csv',
            f'all_books_k{k}_cluster{cluster}.csv'
        ])
    expected_files.extend([
        f'cluster_profiles_k{k}.csv',
        f'thematic_spread_k{k}.csv',
        f'cluster_assignments_k{k}.csv'
    ])

print("\nVerifying presence of all expected output files:")
for file in expected_files:
    file_path = os.path.join(output_dir, file)
    if os.path.exists(file_path):
        print(f"✔️ {file} exists.")
    else:
        print(f"❌ {file} is missing.")


Verifying presence of all expected output files:
✔️ top_30_topics_k3_cluster0.csv exists.
✔️ all_topics_k3_cluster0.csv exists.
✔️ top_5_books_k3_cluster0.csv exists.
✔️ all_books_k3_cluster0.csv exists.
✔️ top_30_topics_k3_cluster1.csv exists.
✔️ all_topics_k3_cluster1.csv exists.
✔️ top_5_books_k3_cluster1.csv exists.
✔️ all_books_k3_cluster1.csv exists.
✔️ top_30_topics_k3_cluster2.csv exists.
✔️ all_topics_k3_cluster2.csv exists.
✔️ top_5_books_k3_cluster2.csv exists.
✔️ all_books_k3_cluster2.csv exists.
✔️ cluster_profiles_k3.csv exists.
✔️ thematic_spread_k3.csv exists.
✔️ cluster_assignments_k3.csv exists.
✔️ top_30_topics_k4_cluster0.csv exists.
✔️ all_topics_k4_cluster0.csv exists.
✔️ top_5_books_k4_cluster0.csv exists.
✔️ all_books_k4_cluster0.csv exists.
✔️ top_30_topics_k4_cluster1.csv exists.
✔️ all_topics_k4_cluster1.csv exists.
✔️ top_5_books_k4_cluster1.csv exists.
✔️ all_books_k4_cluster1.csv exists.
✔️ top_30_topics_k4_cluster2.csv exists.
✔️ all_topics_k4_cluster2.c

In [96]:
# Comparative Analysis

# Initialize DataFrames for comparative analysis
comparative_profiles = pd.DataFrame()
comparative_thematic = pd.DataFrame()

for k in cluster_range:
    # Load Cluster Profiles
    profiles_path = os.path.join(output_dir, f'cluster_profiles_k{k}.csv')
    profiles_df = pd.read_csv(profiles_path)
    profiles_df['k'] = k
    comparative_profiles = pd.concat([comparative_profiles, profiles_df], ignore_index=True)

    # Load Thematic Spread
    thematic_path = os.path.join(output_dir, f'thematic_spread_k{k}.csv')
    thematic_df = pd.read_csv(thematic_path)
    thematic_df['k'] = k
    comparative_thematic = pd.concat([comparative_thematic, thematic_df], ignore_index=True)

# Save Comparative Profiles
comparative_profiles_output = os.path.join(output_dir, 'comparative_cluster_profiles.csv')
comparative_profiles.to_csv(comparative_profiles_output, index=False)
print(f"Saved comparative cluster profiles to '{comparative_profiles_output}'.")
print(f"Sample comparative cluster profiles:")
display(comparative_profiles.head())

# Save Comparative Thematic Spread
comparative_thematic_output = os.path.join(output_dir, 'comparative_thematic_spread.csv')
comparative_thematic.to_csv(comparative_thematic_output, index=False)
print(f"Saved comparative thematic spread to '{comparative_thematic_output}'.")
print(f"Sample comparative thematic spread:")
display(comparative_thematic.head())


Saved comparative cluster profiles to '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/comparative_cluster_profiles.csv'.
Sample comparative cluster profiles:


,Cluster,Topic,Topic_Label,Mean_Probability,Std_Deviation,Cluster_Size,Cluster_Percentage,k
0,0,Topic_72,Love and Affection,56.959680,7.831732,55,52.380952,3
1,0,Topic_70,Leaving and Letting Go,55.085835,7.906873,55,52.380952,3
2,0,Topic_201,Happiness and Gratitude,53.977300,7.330659,55,52.380952,3
3,0,Topic_102,Pleasure and Seduction,53.503047,7.149895,55,52.380952,3
4,0,Topic_175,Moments and Realizations,52.148115,7.278969,55,52.380952,3


Saved comparative thematic spread to '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/comparative_thematic_spread.csv'.
Sample comparative thematic spread:


,Cluster,Topic,Min_Probability,Max_Probability,Median_Probability,Std_Deviation,k
0,0,Topic_72,42.246764,75.136001,57.463841,7.831732,3
1,0,Topic_70,42.079282,70.921499,53.589912,7.906873,3
2,0,Topic_201,41.231069,68.249518,53.141410,7.330659,3
3,0,Topic_102,41.481980,70.952675,52.413939,7.149895,3
4,0,Topic_175,39.748322,65.052580,51.047829,7.278969,3


# Stat

In [99]:
comparative_profiles = pd.read_csv('/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/comparative_cluster_profiles.csv')

In [100]:
cluster_sizes = comparative_profiles[['k', 'Cluster', 'Cluster_Size', 'Cluster_Percentage']].drop_duplicates()
print("Cluster Sizes and Percentages:")
display(cluster_sizes)

Cluster Sizes and Percentages:


,k,Cluster,Cluster_Size,Cluster_Percentage
0,3,0,55,52.380952
30,3,1,20,19.047619
60,3,2,30,28.571429
90,4,0,42,40.000000
120,4,1,31,29.523810
150,4,2,26,24.761905
180,4,3,6,5.714286
210,5,0,30,28.571429
240,5,1,15,14.285714
270,5,2,24,22.857143


In [101]:
for k in sorted(comparative_profiles['k'].unique()):
    print(f"\nTop 30 Topics for k={k}:")
    clusters_k = comparative_profiles[comparative_profiles['k'] == k]
    for cluster in sorted(clusters_k['Cluster'].unique()):
        print(f"\nCluster {cluster}:")
        cluster_data = clusters_k[clusters_k['Cluster'] == cluster]
        # Sort topics by mean probability descending
        cluster_data = cluster_data.sort_values(by='Mean_Probability', ascending=False)
        # Display the top 30 topics
        display(cluster_data[['Topic', 'Topic_Label', 'Mean_Probability', 'Std_Deviation']].head(30))


Top 30 Topics for k=3:

Cluster 0:


,Topic,Topic_Label,Mean_Probability,Std_Deviation
0,Topic_72,Love and Affection,56.959680,7.831732
1,Topic_70,Leaving and Letting Go,55.085835,7.906873
2,Topic_201,Happiness and Gratitude,53.977300,7.330659
3,Topic_102,Pleasure and Seduction,53.503047,7.149895
4,Topic_175,Moments and Realizations,52.148115,7.278969
5,Topic_129,Breaking and Letting Go,51.073280,7.271985
6,Topic_0,Expressions of Joy and Amusement,50.836480,6.574115
7,Topic_117,Possession and Desire,50.482750,7.913357
8,Topic_36,Nods and Agreement,50.172346,6.841607
9,Topic_12,Nightlife and Events,49.672491,6.715273



Cluster 1:


,Topic,Topic_Label,Mean_Probability,Std_Deviation
30,Topic_70,Leaving and Letting Go,83.676551,11.296283
31,Topic_72,Love and Affection,83.332764,10.283198
32,Topic_102,Pleasure and Seduction,81.832056,13.278993
33,Topic_117,Possession and Desire,81.322740,14.630324
34,Topic_201,Happiness and Gratitude,81.109163,11.417902
35,Topic_129,Breaking and Letting Go,79.294983,11.628158
36,Topic_175,Moments and Realizations,79.119320,11.056705
37,Topic_0,Expressions of Joy and Amusement,77.696095,14.997231
38,Topic_14,Conversations and Communication,76.266857,13.108001
39,Topic_36,Nods and Agreement,75.582944,11.561559



Cluster 2:


...

,Topic,Topic_Label,Mean_Probability,Std_Deviation
270,Topic_72,Love and Affection,23.908134,8.859000
271,Topic_70,Leaving and Letting Go,22.774518,8.142222
272,Topic_201,Happiness and Gratitude,22.141736,7.912488
273,Topic_102,Pleasure and Seduction,21.962669,7.332418
274,Topic_175,Moments and Realizations,21.590848,7.443665
275,Topic_129,Breaking and Letting Go,20.859708,7.408546
276,Topic_36,Nods and Agreement,20.610543,7.323531
277,Topic_0,Expressions of Joy and Amusement,20.556591,7.056362
278,Topic_167,Romance and Feelings,20.214329,7.099354
279,Topic_12,Nightlife and Events,20.187967,6.983725



Cluster 3:


,Topic,Topic_Label,Mean_Probability,Std_Deviation
300,Topic_117,Possession and Desire,102.350500,9.802082
301,Topic_102,Pleasure and Seduction,100.731692,10.456139
302,Topic_0,Expressions of Joy and Amusement,99.911051,10.112502
303,Topic_70,Leaving and Letting Go,99.523871,8.895314
304,Topic_201,Happiness and Gratitude,96.416885,9.100102
305,Topic_72,Love and Affection,96.254246,9.659225
306,Topic_129,Breaking and Letting Go,95.972706,9.235776
307,Topic_175,Moments and Realizations,94.765338,8.226904
308,Topic_14,Conversations and Communication,94.673497,8.978944
309,Topic_36,Nods and Agreement,92.203727,8.624458



Cluster 4:


,Topic,Topic_Label,Mean_Probability,Std_Deviation
330,Topic_72,Love and Affection,62.530946,4.966763
331,Topic_70,Leaving and Letting Go,60.544606,5.709485
332,Topic_201,Happiness and Gratitude,59.159826,5.006449
333,Topic_102,Pleasure and Seduction,58.578761,4.805507
334,Topic_175,Moments and Realizations,57.448501,4.687281
335,Topic_129,Breaking and Letting Go,56.354186,4.703318
336,Topic_0,Expressions of Joy and Amusement,55.801518,3.778632
337,Topic_117,Possession and Desire,55.229289,7.074059
338,Topic_36,Nods and Agreement,55.175505,4.095220
339,Topic_12,Nightlife and Events,54.509775,4.499091


# Statistical Analysis

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd

print(f"Pandas version: {pd.__version__}")

Pandas version: 2.2.2


In [ ]:
# Comparative Metrics Across Clusters - Shared Themes Using Pearson Correlation

from scipy.stats import pearsonr
import pandas as pd
import os

# Initialize a list to store Pearson correlation results
correlation_results = []

for k in cluster_range:
    print(f"\nCalculating Pearson Correlation for shared themes in k={k}...")

    # Load cluster profiles for k
    profiles_path = os.path.join(output_dir, f'cluster_profiles_k{k}.csv')
    profiles_df = pd.read_csv(profiles_path)

    # Select only the necessary columns to avoid including grouping columns
    profiles_subset = profiles_df[['Cluster', 'Topic', 'Mean_Probability']]

    # Get the top 30 topics for each cluster
    top_n = 30
    # Remove 'include_groups=False' to avoid the TypeError
    top_topics_df = profiles_subset.groupby('Cluster', group_keys=False).apply(
        lambda x: x.nlargest(top_n, 'Mean_Probability')
    )

    # Get the list of unique clusters
    clusters = top_topics_df['Cluster'].unique()

    # Iterate through each unique pair of clusters
    for i in range(len(clusters)):
        for j in range(i+1, len(clusters)):
            cluster_a = clusters[i]
            cluster_b = clusters[j]

            # Extract mean probabilities for the top 30 topics in each cluster
            topics_a = top_topics_df[top_topics_df['Cluster'] == cluster_a].set_index('Topic')['Mean_Probability']
            topics_b = top_topics_df[top_topics_df['Cluster'] == cluster_b].set_index('Topic')['Mean_Probability']

            # Find common topics between the two clusters
            common_topics = topics_a.index.intersection(topics_b.index)

            if len(common_topics) < 2:
                print(f"Not enough common topics between Cluster {cluster_a} and Cluster {cluster_b} for k={k}. Skipping...")
                continue

            # Extract the probabilities for the common topics
            prob_a = topics_a[common_topics]
            prob_b = topics_b[common_topics]

            # Calculate Pearson correlation
            corr_coef, p_val = pearsonr(prob_a, prob_b)

            # Append the results to the list
            correlation_results.append({
                'k': k,
                'Cluster_A': cluster_a,
                'Cluster_B': cluster_b,
                'Pearson_Correlation': corr_coef,
                'P_Value': p_val
            })

    # Display the correlations for current k
    correlations_df = pd.DataFrame([res for res in correlation_results if res['k'] == k])
    print(f"\nPearson Correlation Coefficients for k={k}:")
    display(correlations_df[['Cluster_A', 'Cluster_B', 'Pearson_Correlation', 'P_Value']])

# Convert the complete list to a DataFrame
shared_themes_df = pd.DataFrame(correlation_results)

# Save the shared themes correlations to CSV
shared_themes_output = os.path.join(output_dir, 'shared_themes_pearson_correlation.csv')
shared_themes_df.to_csv(shared_themes_output, index=False)
print(f"\nSaved shared themes Pearson correlation coefficients to '{shared_themes_output}'.")

# Additional Check: Verify the contents of the CSV
print("\nVerifying the contents of 'shared_themes_pearson_correlation.csv':")
shared_themes_preview = pd.read_csv(shared_themes_output).head()
display(shared_themes_preview)


Calculating Pearson Correlation for shared themes in k=3...

Pearson Correlation Coefficients for k=3:


,Cluster_A,Cluster_B,Pearson_Correlation,P_Value
0,0,1,0.961984,9.523581e-17
1,0,2,0.991741,1.269707e-25
2,1,2,0.927417,1.306236e-12



Calculating Pearson Correlation for shared themes in k=4...

Pearson Correlation Coefficients for k=4:


,Cluster_A,Cluster_B,Pearson_Correlation,P_Value
0,0,1,0.973841,6.570738e-19
1,0,2,0.994438,6.411918e-29
2,0,3,0.835185,1.113465e-07
3,1,2,0.964234,4.235807e-17
4,1,3,0.876965,4.180925e-09
5,2,3,0.797634,1.062801e-06



Calculating Pearson Correlation for shared themes in k=5...

Pearson Correlation Coefficients for k=5:


,Cluster_A,Cluster_B,Pearson_Correlation,P_Value
0,0,1,0.969128,2.470889e-17
1,0,2,0.981609,5.912423e-21
2,0,3,0.828189,3.213007e-07
3,0,4,0.995526,3.308764e-29
4,1,2,0.931587,1.738360e-12
5,1,3,0.861084,3.306805e-08
6,1,4,0.970265,3.627208e-18
7,2,3,0.747472,2.696304e-05
8,2,4,0.985348,1.678187e-21
9,3,4,0.822111,4.642600e-07



Saved shared themes Pearson correlation coefficients to '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/shared_themes_pearson_correlation.csv'.

Verifying the contents of 'shared_themes_pearson_correlation.csv':


,k,Cluster_A,Cluster_B,Pearson_Correlation,P_Value
0,3,0,1,0.961984,9.523581e-17
1,3,0,2,0.991741,1.269707e-25
2,3,1,2,0.927417,1.306236e-12
3,4,0,1,0.973841,6.570738e-19
4,4,0,2,0.994438,6.411918e-29


In [ ]:
from statsmodels.stats.multitest import multipletests
import numpy as np

# Load the shared themes Pearson correlation data
shared_themes_path = os.path.join(output_dir, 'shared_themes_pearson_correlation.csv')
shared_themes_df = pd.read_csv(shared_themes_path)

# Initialize a column for adjusted p-values
shared_themes_df['Adjusted_P_Value'] = np.nan

# Apply Benjamini-Hochberg correction per k
for k in cluster_range:
    mask = shared_themes_df['k'] == k
    pvals = shared_themes_df.loc[mask, 'P_Value']

    # Check if there are p-values to adjust
    if pvals.empty:
        print(f"No p-values to adjust for k={k}.")
        continue

    # Apply Benjamini-Hochberg correction
    reject, pvals_corrected, _, _ = multipletests(pvals, method='fdr_bh')

    # Assign the corrected p-values back to the DataFrame
    shared_themes_df.loc[mask, 'Adjusted_P_Value'] = pvals_corrected

# Save the corrected correlations to CSV
corrected_shared_themes_output = os.path.join(output_dir, 'shared_themes_pearson_correlation_corrected.csv')
shared_themes_df.to_csv(corrected_shared_themes_output, index=False)
print(f"Saved corrected shared themes Pearson correlation coefficients to '{corrected_shared_themes_output}'.")

# Additional Check: Verify the contents of the corrected CSV
print("\nVerifying the contents of 'shared_themes_pearson_correlation_corrected.csv':")
corrected_shared_themes_preview = pd.read_csv(corrected_shared_themes_output).head()
display(corrected_shared_themes_preview)

Saved corrected shared themes Pearson correlation coefficients to '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/shared_themes_pearson_correlation_corrected.csv'.

Verifying the contents of 'shared_themes_pearson_correlation_corrected.csv':


,k,Cluster_A,Cluster_B,Pearson_Correlation,P_Value,Adjusted_P_Value
0,3,0,1,0.961984,9.523581e-17,1.428537e-16
1,3,0,2,0.991741,1.269707e-25,3.809121e-25
2,3,1,2,0.927417,1.306236e-12,1.306236e-12
3,4,0,1,0.973841,6.570738e-19,1.971221e-18
4,4,0,2,0.994438,6.411918e-29,3.847151e-28


In [ ]:
# Calculating Silhouette Scores for Each k

from sklearn.metrics import silhouette_score
import pandas as pd
import os
import numpy as np

# Initialize a list to store silhouette scores
silhouette_results = []

for k in cluster_range:
    print(f"\nCalculating Silhouette Score for k={k}...")

    # Load cluster assignments
    cluster_assignments_path = os.path.join(output_dir, f'cluster_assignments_k{k}.csv')
    cluster_assignments_df = pd.read_csv(cluster_assignments_path)

    # Extract cluster labels
    cluster_labels = cluster_assignments_df['Cluster']

    # Calculate silhouette score
    try:
        score = silhouette_score(df_scaled, cluster_labels)
        silhouette_results.append({
            'k': k,
            'Silhouette_Score': score
        })
        print(f"Silhouette Score for k={k}: {score:.4f}")
    except Exception as e:
        print(f"Error calculating silhouette score for k={k}: {e}")
        silhouette_results.append({
            'k': k,
            'Silhouette_Score': np.nan
        })

# Convert the list to a DataFrame
silhouette_scores_df = pd.DataFrame(silhouette_results)

# Save the silhouette scores to CSV
silhouette_scores_output = os.path.join(output_dir, 'silhouette_scores.csv')
silhouette_scores_df.to_csv(silhouette_scores_output, index=False)
print(f"\nSaved silhouette scores to '{silhouette_scores_output}'.")

# Additional Check: Verify the contents of the silhouette scores CSV
print("\nVerifying the contents of 'silhouette_scores.csv':")
silhouette_scores_preview = pd.read_csv(silhouette_scores_output).head()
display(silhouette_scores_preview)


Calculating Silhouette Score for k=3...
Silhouette Score for k=3: 0.5331

Calculating Silhouette Score for k=4...
Silhouette Score for k=4: 0.4907

Calculating Silhouette Score for k=5...
Silhouette Score for k=5: 0.4878

Saved silhouette scores to '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/silhouette_scores.csv'.

Verifying the contents of 'silhouette_scores.csv':


,k,Silhouette_Score
0,3,0.533099
1,4,0.490740
2,5,0.487840


In [ ]:
# Function to verify CSV file contents
def verify_csv(file_path, expected_columns, num_rows=5):
    """
    Verifies the contents of a CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.
    - expected_columns (list): List of expected column names.
    - num_rows (int): Number of rows to display for verification.

    Returns:
    - None
    """
    if not os.path.exists(file_path):
        print(f"File '{file_path}' does not exist.")
        return

    df = pd.read_csv(file_path)
    actual_columns = df.columns.tolist()

    # Check for expected columns
    missing_columns = set(expected_columns) - set(actual_columns)
    if missing_columns:
        print(f"File '{file_path}' is missing columns: {missing_columns}")
    else:
        print(f"All expected columns are present in '{file_path}'.")

    # Display the first few rows
    print(f"\nFirst {num_rows} rows of '{file_path}':")
    display(df.head(num_rows))

# Example Usage:

# Verify 'shared_themes_pearson_correlation.csv'
verify_csv(
    file_path=os.path.join(output_dir, 'shared_themes_pearson_correlation.csv'),
    expected_columns=['k', 'Cluster_A', 'Cluster_B', 'Pearson_Correlation', 'P_Value']
)

# Verify 'shared_themes_pearson_correlation_corrected.csv'
verify_csv(
    file_path=os.path.join(output_dir, 'shared_themes_pearson_correlation_corrected.csv'),
    expected_columns=['k', 'Cluster_A', 'Cluster_B', 'Pearson_Correlation', 'P_Value', 'Adjusted_P_Value']
)

# Verify 'silhouette_scores.csv'
verify_csv(
    file_path=os.path.join(output_dir, 'silhouette_scores.csv'),
    expected_columns=['k', 'Silhouette_Score']
)

All expected columns are present in '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/shared_themes_pearson_correlation.csv'.

First 5 rows of '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/shared_themes_pearson_correlation.csv':


,k,Cluster_A,Cluster_B,Pearson_Correlation,P_Value
0,3,0,1,0.961984,9.523581e-17
1,3,0,2,0.991741,1.269707e-25
2,3,1,2,0.927417,1.306236e-12
3,4,0,1,0.973841,6.570738e-19
4,4,0,2,0.994438,6.411918e-29


All expected columns are present in '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/shared_themes_pearson_correlation_corrected.csv'.

First 5 rows of '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/shared_themes_pearson_correlation_corrected.csv':


,k,Cluster_A,Cluster_B,Pearson_Correlation,P_Value,Adjusted_P_Value
0,3,0,1,0.961984,9.523581e-17,1.428537e-16
1,3,0,2,0.991741,1.269707e-25,3.809121e-25
2,3,1,2,0.927417,1.306236e-12,1.306236e-12
3,4,0,1,0.973841,6.570738e-19,1.971221e-18
4,4,0,2,0.994438,6.411918e-29,3.847151e-28


All expected columns are present in '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/silhouette_scores.csv'.

First 5 rows of '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/silhouette_scores.csv':


,k,Silhouette_Score
0,3,0.533099
1,4,0.490740
2,5,0.487840


In [ ]:
import pandas as pd
import os

# Define output directory
output_dir = '/content/drive/MyDrive/BERTTopic_Models/paraphrase-MiniLM-L6-v2/results/clustering_outputs/'

# Specify a k value to inspect
k = 3
cluster_profiles_path = os.path.join(output_dir, f'cluster_profiles_k{k}.csv')

# Load the CSV
cluster_profiles_df = pd.read_csv(cluster_profiles_path)

# Display the first few rows
display(cluster_profiles_df.head())

,Cluster,Topic,Topic_Label,Mean_Probability,Std_Deviation,Cluster_Size,Cluster_Percentage
0,0,Topic_72,Love and Affection,56.959680,7.831732,55,52.380952
1,0,Topic_70,Leaving and Letting Go,55.085835,7.906873,55,52.380952
2,0,Topic_201,Happiness and Gratitude,53.977300,7.330659,55,52.380952
3,0,Topic_102,Pleasure and Seduction,53.503047,7.149895,55,52.380952
4,0,Topic_175,Moments and Realizations,52.148115,7.278969,55,52.380952
